# Introduction to the human metabolic reconstruction and FBA and FVA

**Authors**: Thierry D.G.A Mondeel, Stefania Astrologo, Ewelina Weglarz-Tomczak & Hans V. Westerhoff <br/>
University of Amsterdam <br/>
2016 - 2019

**Note:** Some of the material in this tutorial is inspired by and adapted from the cell factory design course from The Novo Nordisk Foundation Center for Biosustainability https://biosustain.github.io/cell-factory-design-course/

---

**Questions we will explore in this notebook**
- How do I explore the content (metabolites, reactions, genes, …) of the genome-wide human metabolic map?
- How do I figure out what kinds of metabolic fluxes are possible at steady state?
- What kind of questions can I answer by performing computational "experiments"?
- How can we understand the robustness of such metabolic flux patterns?

**Objectives**
- Understand the basic data structures of metabolic maps
- Set a biological aim and see whether that aim may be achieved in a human metabolic map 
- Manipulate media conditions and knocking out reactions/genes
- Find essential reactions for survival and fitness

## Refresher: the human metabolic reconstruction: Recon 2 and Recon 3D

See the publication: [Recon 2 (2013)](http://doi.org/10.1038/nbt.2488) and [Recon 3D (2018)](https://www.nature.com/articles/nbt.4072).

<span style="color:red">**Assignment (5 min):**</span> Read the abstract of the 2013 publication and the first two paragraphs of the introduction.

## Loading the human metabolic map
<span style="color:red">**Assignment (1 sec):**</span> Execute the cell below. This cell loads the cobrapy computational toolbox and loads the human metabolic map. **The specifics of the code do not matter**

In [ ]:
# load the FBA module
import cobra
from cobra.flux_analysis import pfba

# load the table and data analysis module
import pandas as pd # for tables
pd.set_option('display.max_colwidth', -1) # don't constrain the content of the tables
pd.options.display.max_rows = 9999

# load the popular plotting library
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

# show all output in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon3D with simplified medium for this tutorial
M = cobra.io.load_json_model('./models/Recon3D_301/Recon3DModel_301_simple_medium.json')

# Copy the model, keep M as the original unaltered model
model = M.copy() 

# What is a human metabolic map (reconstruction) really?
Above we loaded the human metabolic reconstruction. An easy way to think about such reconstructions is that they are maps, very much like road maps such as: [Google Maps](http://maps.google.com). Below we depict an example of how to walk from the O2 building to the main VU campus.

![Google Maps example](./images/Google_maps_VU_O2.png)

You can think of a metabolic map as having three key components:
1. Metabolites
    > Places you want to go (like the VU campus)
2. Reactions
    > Roads that connect where you are to where you want to go (like the Boelelaan)
3. Reactions are coupled to gene(s) that encode the enzyme catalyzing the reaction.
    > Do I need a car to take this road? Or a bike? Or either? Is their construction on the road (i.e. no traffic, i.e. no gene expression), is there high or low traffic capacity on this road (i.e. low or high enzyme expression)

We can interact with the map through 3 different kinds of object: metabolites, reactions and genes. We will look at these below.

## Accessing the contents of the map: metabolites, reactions and genes
The model for the human metabolic network contains many different genes, reactions and metabolites.

> You can access each of these three sets (genes, reactions and metabolites) through 'dot' notation: 
>
> e.g. model.genes

<span style="color:red">**Assignment (5 min):**</span> Find out how many genes, reactions and metabolites the model  contains. **Tip:** use the len() function and use the dot notation (e.g. model.genes) shown above. We already did the genes for you as an example.

In [ ]:
# calculate the number of genes here
len(model.genes)

# calculate the number of reactions here

# calculate the number of metabolites here


<span style="color:red">**Assignment (2 min):**</span> Why are there so few genes associated with this metabolic map? What about all the other human genes? How many genes do humans actually have in total? 

See the abstract of: http://doi.org/10.1126/science.1058040

### Metabolites

One can access a specific metabolite using dot notation.

> model.metabolites.METABOLITE_ID

<span style="color:red">**Assignment (1 min):**</span>
* Look at the properties of the metabolite ATP, below. 
* Notice the dot notation used to access it
* What percentage of reactions in the map does ATP partake in?

In [ ]:
model.metabolites.atp_c

#### Finding metabolites
If you are not sure which ID the metabolite you are looking has, you can use a query to look for it. Suppose we want to find lactate:

In [ ]:
model.metabolites.query('lactate','name')

The above doesn't look very user friendly. Let's build a nice table to view these metabolites. You don't need to understand the code below but execute the cell and look at its output.

In [ ]:
df = pd.DataFrame(columns=['ID',"Name","Compartment","Formula"]) # start empty table

for i, metabolite in enumerate(model.metabolites.query('lactate','name')):
    df.loc[i] = [metabolite.id, metabolite.name, metabolite.compartment, metabolite.formula]

df

<span style="color:red">**Assignment (3 min):**</span> 
* Search for another metabolite of your interest and look at its properties. 
* To start, copy the code from the cell above and modify the search term

#### Properties of metabolites
Metabolites are associated with compartments in the cell. Glyceraldehyde 3-phosphate (g3p_c) is associated with the c (Cytosol) compartment.


In [ ]:
model.metabolites.g3p_c.compartment

The metabolites in the human metabolic reconstruction are categorized in various compartments. 

In [ ]:
list(model.compartments.keys())

<span style="color:red">**Assignment (3 min):**</span> Can you identify which compartments the identifiers point to? Do you know the biological role of all these compartments?

See here for what biochemical functions are associated with each compartment: http://www.ncbi.nlm.nih.gov/books/NBK26907/

**Answers:**
C = cytosol, m = mitochondria, i = intermembrane space, x = peroxisome, l = lysosome, g = golgi apparatus, e = extracellular

<span style="color:red">**Assignment (3 min):**</span> Some metabolites (like Glucose for example) can be associated with multiple compartments. Do they participate in the same reactions? Look at the listed reactions in the tables below.

In [ ]:
model.metabolites.glc_D_c
model.metabolites.glc_D_g

The full name of the metabolite is available via the .name attribute.

In [ ]:
model.metabolites.glc_D_c.name

One can look up the molecular formula of glucose and its weight.

The .elements attribute returns a dictionary representation of the formula.

In [ ]:
model.metabolites.glc_D_c.formula
model.metabolites.glc_D_c.formula_weight
model.metabolites.glc_D_c.elements

### Reactions

Metabolites are not isolated things. They participate in reactions as substrates and products.


Reactions have similar properties as metabolites. Below we show pyruvate kinase, like we showed ATP before.

**Note:** metabolite identifiers are usually lowercase, whereas reaction identifiers are usually uppercase.

In [ ]:
model.reactions.PYK

Let's take a closer look at the reactions associated with Glyceraldehyde 3-phosphate (g3p). Reactions like metabolites have both a short ID, a longer name attribute and an attribute called reaction that contains the chemical reaction equation. Lets see all of these for the reactions g3p engages in.

In [ ]:
df = pd.DataFrame(columns=['ID',"Name",'Reaction','Subsystem']) # start empty table

# add each reaction to the table
for i, reaction in enumerate(model.metabolites.g3p_c.reactions):
    df.loc[i] = [reaction.id,reaction.name, reaction.reaction, reaction.subsystem]
df

#### Reaction bounds
In constraint-based (FBA) models reactions are associated with two bounds, a lower bound and an upper bound. These constrain the amount of flux that is allowed to run through a certain reaction. 

The most important way these are used is to take into account thermodynamics of reactions. If a certain reaction has a very high **negative** $\Delta G^{'0}$ then this reaction is practically irreversible. As such its lower bound is often made zero, meaning that the flux cannot run in the backward direction because it goes against thermodynamics. This makes the model predictions more realistic.

##### If you need a refresher on thermodynamics 
This page has all you need to know: https://www.khanacademy.org/science/chemistry/thermodynamics-chemistry/gibbs-free-energy/a/gibbs-free-energy-and-spontaneity

Let's look at an example: pyruvate kinase has a lower bound of zero in our model, i.e. it is irreversible in the direction of pyruvate.

In [ ]:
model.reactions.PYK.name
model.reactions.PYK.reaction
model.reactions.PYK.bounds

<span style="color:red">**Assignment (3 min):**</span> Check that this irreversibility makes sense thermodynamically. Go to: http://equilibrator.weizmann.ac.il/ and type in "pyruvate kinase". This will return multiple variations but click on the reaction involving ATP.

Look at the estimates for the $\Delta G^{'0}$. Does this match with the model (in terms of the irreversibility)? 

To get a sense of the size of this dG, compare it with $R*T = 8.3*0.298 = 2.4734~kJ/mole$, which is the energy due to random thermal motion. The dG is about 10x higher than this. 

### Genes and their link to reactions
Remember that the metabolic network is made up of reactions that link together metabolites. The second layer of the model consists of genes coupling to reactions. 

Glyceraldehyde-3-phosphate dehydrogenase is associated with two genes.

In [ ]:
model.reactions.GAPD.gene_reaction_rule

These gene identifiers refer to the NCBI database: https://www.ncbi.nlm.nih.gov/gene/?term=2597 

Note that the OR logic in the gene rule, implies that only one of the isoenzymes (encoded by either gene) is needed for this reaction. 

<span style="color:red">**Assignment (3 min):**</span> Is this annotated correctly. Why two genes? If you look this up in NCBI, remove the .1 from the gene identifier.

### The medium
The next key component of the model is the "medium", i.e. the set of metabolites that is allowed to be taken up into the cell. For human cells this should entail the essential amino acids, glucose, oxygen etc. The way this is done in the model is by having so-called "exchange reactions" that transport such a metabolite into the extracellular space (the "e" compartment we came across above) from the outside world. Typically each of these reactions start with 'EX_' which stands for exchange reaction. The word exchange refers to the fact that this is an exchange of metabolites with the environment or the biofluids surrounding the cell. 

Exchange reactions are defined as: X <=> . So a negative (to the left) flux means uptake of metabolite X into the system. Positive (to the right) flux would mean X is produced by the cell. This would typically be the case for lactate or CO2 etc. 

<span style="color:red">**Assignment (3 min):**</span> Make sure you understand the difference between an exchange reaction and a transport reaction. They are not the same thing!

Exchange $$X <=> $$ Transport $$ X_c <=> X_e$$

Below we print the medium of the model. The table shows the bound on each such inward flux. The bounds on these fluxes are actually negative but are here shown as positive values. 

In [ ]:
pd.DataFrame(list(model.medium.items()),columns=['Reaction','Inward flux']).sort_values('Reaction').set_index('Reaction')

<span style="color:red">**Assignment (3 min):**</span> Would you say this is a reasonable medium for a human cell to live in? What are the limiting medium components currently? Is there an oxygen or carbon limitation? What is/are the carbon source(s)? 

# Performing Flux Balance Analysis

## What is FBA?
We will use the constraint-based modeling technique called flux balance analysis ([FBA](http://doi.org/10.1038/nbt.1614)). 

<span style="color:red">**Assignment (5 min):**</span> Open [this publication](http://doi.org/10.1038/nbt.1614) and read the first paragraph of the introduction.

<span style="color:red">**Assignment (10 min):**</span> Make sure you globally understand the workflow presented in the images  and the accompanying text below.

Look at the image below from Orth et al. FBA starts from a metabolic reconstruction of the network: i.e. listing all possible metabolic functions of an organism (**Step A**). 

FBA primarily makes use of the stoichiometry matrix $S$ (**Step B**). This matrix is of size $m \times r$ where $m$ is the number of species or metabolites in the system and $r$ is the number of reactions. Every row of $S$ specifies for a specific metabolite in what quantity it partakes in each reaction. Therefore, each element $(i,j)$ of $S$ contains the stoichiometric coefficient of metabolite $i$ in reaction $j$.

Given the metabolic network as summarized by the stoichiometry matrix, FBA aims to computationally calculate a "flux distribution", meaning the fluxes through all reactions in the model, **at steady-state**. Steady state implies that all fluxes into and out of each metabolite in the network sum up to zero (**Step C**). In other words, the mass-balance must equal zero for each metabolite. Mathematically, the steady state (mass-balance) condition is written as $Sv=0$ (See the bottom image below). 

Each reaction in FBA may be assigned a $V_{max}$ in the forward and backward direction. We write this as "bounds on the flux" as follows: $\alpha_i <= v_i <= \beta_i$ for a particular reaction $i$ (See the bottom image below). If expression data is available for all enzymes, one can set the $\alpha$ and $\beta$ proportional to the expression of the enzyme. Here, for simplicity, we will bound all reactions by the same level $1000$. **Note:** the metabolic reconstruction does already account for reactions that are not reversible due to the thermodynamics by making some $\alpha_i = 0$. 

Typically, with FBA we predict a flux distribution that is optimal according to a specified "objective function", i.e. a purpose or aim the cell is after (**Step D**). In our case, the objective function will usually be the flux through the biomass reaction. The biomass reaction is a proxy for cell growth. This reaction contains (when known) experimentally determined ratio's of metabolites that make up the composition of a cell.

Ultimately, FBA then computes the flux distribution that satisfies:
- (1) the steady-state condition: based on the assumption that metabolism occurs on a fast time-scale compared to gene regulatory events and thus reaction rates are constant 
- (2) thermodynamic feasibility, i.e. some reactions are known to be irreversible, 
- (3) maximal flux constraints when these are known
- (4) maximizes some objective like growth

![Orth et al., 2010 Figure 2](./images/Orth_2010_Fig2.gif)

![Orth et al., 2010 Figure 1](./images/Orth_2010_Fig1.gif)

### What you will do
We will perform in-silico experiments with genetic manipulations by changing the lower and upper bounds for the reactions. By setting both the lower bound $\alpha_k$ and the upper bound $\beta_k$ to zero we in effect knock out a reaction. Similarly, by setting it to a specific value we can fix a certain flux level for a specific reaction.

### What is our in-silico cell optimizing? 
Let us first check what objective is set for our model.

In [ ]:
cobra.util.linear_reaction_coefficients(model)

So the objective is biomass.

<span style="color:red">**Assignment (3 min):**</span> Is optimal growth a realistic assumption for human cells? For bacterial cells? Discuss... 

### Doing FBA is simple...
Actually performing FBA with cobrapy once you have loaded a model is really easy and fast. 

In [ ]:
solution = model.optimize()

"solution" now contains the result of our simulation. Let's see what we can do with this. The most important attributes here are:
- status => did we find an optimial solution?
- objective_value => what is the biomass flux?
- fluxes => the flux through every reaction in the model solution. Below we print the GAPDH flux for example.

In [ ]:
solution.status
solution.objective_value
solution.fluxes['GAPD']

A very nice summary of what goes in and out of the cell in this solution can be viewed with the summary attribute of the model object.

In [ ]:
model.summary()

<span style="color:red">**Assignment (3 min):**</span> Investigate the metabolites that are produced (i.e. in the OUT FLUXES column). Are these what you would have expected? 

<span style="color:red">**Assignment (3 min):**</span> What would happen if we force the cell to produce lactate (i.e. to ferment) and not respire? Try this by setting the **lower_bound** of the exchange reaction of lactate 'EX_lac_D_e' or 'EX_lac_L_e' to a positive number. Lactate is a C3, and we are taking up 1 unit of glucose C6) so set the minimal lactate production flux (i.e. its lower bound) equal to 2. And block CO2 production by setting its exchange reaction to zero. **We did this below for you already**.

What new products have been produced? Did something change in the uptake fluxes? Did the growth rate remain the same? Are there still secondary carbon exits?

In [ ]:
model = M.copy()

# We change the lactate lower_bound here
model.reactions.EX_lac_D_e.lower_bound = 2
model.reactions.EX_co2_e.upper_bound = 0
model.reactions.EX_hco3_e.upper_bound = 0 # equivalent to CO2 as an exit

solution = model.optimize()
model.summary()

<span style="color:red">**Assignment (3 min):**</span> Now try simulating a case where some of the additional carbon exit pathways, e.g. the one producing propionate are transcriptionally constrained. Implement this by limiting the production by setting the upper bound of their exchange reactions to zero or a low number. We already provided some code below.

What happens to growth? 

In [ ]:
model = M.copy()

model.reactions.EX_lac_D_e.lower_bound = 2
model.reactions.EX_co2_e.upper_bound = 0
model.reactions.EX_hco3_e.upper_bound = 0 # equivalent to CO2 as an exit

model.reactions.EX_ppa_e.upper_bound = 0.2

solution = model.optimize()
model.summary()

<span style="color:red">**Assignment (3 min):**</span> Below use the code we supplied to simulate a case where lactate production is not allowed and CO2 production is required. 

In [ ]:
model = M.copy()

model.reactions.EX_lac_D_e.upper_bound = 0
model.reactions.EX_lac_L_e.upper_bound = 0
model.reactions.EX_co2_e.lower_bound = 6

solution = model.optimize()
model.summary()

<span style="color:red">**Assignment (5 min):**</span> Above you might have realized that the model can grow optimally virtually only producing CO2 and propionate (ppa) as a carbon products. You also observed that it grows optimally by fermentation (lactate production) alone. What does that imply about what is limiting growth? Is it ATP or something else?

## How is ATP generated? 
### When maximizing ATP production using only glucose and o2
Below we first remove the amino acids from the medium in order to just ask for ATP production from glucose and oxygen. 

Then we make the ATP maintenance reaction: atp -> adp + pi, the objective reaction. 

<span style="color:red">**Assignment (3 min):**</span> Before running the cell below. Based on your textbook knowledge: what would you expect the maximum number of ATP to be given 1 unit of glucose and unlimited oxygen? 
How much oxygen would be used for this? 

Now execute the cell and compare the results. 

In [ ]:
model = M.copy()

# block amino acid uptake
for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e','EX_glu_L_e','EX_gln_L_e']:
    model.reactions.get_by_id(rxn).lower_bound = 0

model.objective = model.reactions.ATPM
solution = model.optimize()

model.summary()
print("\nLet's look at cytosolic ATP production\n")
model.metabolites.atp_c.summary()
print("\nLet's look at mitochondrial ATP production \n")
model.metabolites.atp_m.summary()
print("\nLet's look at proton pumping\n")
model.metabolites.h_i.summary()

### When maximizing biomass
Below we do the same again but for biomass as the objective reaction. 

In [ ]:
model = M.copy()

solution = model.optimize()

model.summary()
print("\nLet's look at cytosolic ATP production\n")
model.metabolites.atp_c.summary()
print("\nLet's look at mitochondrial ATP production \n")
model.metabolites.atp_m.summary()
print("\nLet's look at proton pumping\n")
model.metabolites.h_i.summary()

<span style="color:red">**Assignment (3 min):**</span> Reflect for a moment on the differences compared to the ATPM objective. Why is the ATPS4 flux down? Why is the oxygen/glucose ratio down? 

**Answer:** When the objective is biomass production the cell needs to produce more than just ATP: protein, lipids etc. This takes up some of the carbon taken up and therefore less of the carbon can be used to produce ATP. 

# Flux variability analysis (FVA)
**Questions**
* How uniquely determined is the flux distribution returned by flux balance analysis?

**Objectives**
* Learn how to determine flux capacities using flux variability analysis.
* Learn how to generate phenotypic phase planes for different fluxes.
--- 

Flux balance analysis gives you one **optimal** steady state solution for the model you are simulating. Above you saw that you could alter the network, e.g. by blocking one lactate exit, and still get an optimal solution. This means that **the FBA solution is not uniquely determined**. 

## What is flux variability analysis?
(Mahadevan and Schilling, 2003) http://doi.org/10.1016/j.ymben.2003.09.002

Flux variability analysis is closely related to flux balance analysis. 

Remember that FBA calculates one **flux distribution** such that the whole network is at steady state, mass-balanced, thermodynamically feasible and optimal. 

In contrast, FVA calculates for each reaction **the range of feasible flux** that is consistent with these constraints FBA uses. So for a given reaction $v$, FVA will tell you what the minimal and maximal attainable flux is through that reaction while keeping the solution at steady state and optimal. 

Technically, FVA for one reaction, for instance pyruvate kinase, involves once performing FBA with the objective of maximizing the flux through pyruvate kinase and once performing FBA to minimize its flux. It then returns this interval [minimum_feasible flux, maximum_feasible_flux]. 

## Why would we perform FVA? 
* Flux Balance Analysis solutions are not necessariliy unique. Flux Variablity Analysis is a good tool for finding alternative flux patterns while achieving the same growth rate. For instance, the possibility for the cell to grow using fermentation or respiration. We could judge this by looking at O2 uptake, lactate secretion, proton pumping etc. If we find an interval of fluxes for these reactions using FVA it indicates the ability for the cells to shift 

## How to perform FVA?
cobra.io.flux_variability_analysis calculates all the minimum and maximum fluxes that all reactions in a model can attain.

In [ ]:
model = M.copy()

# a short list of reactions to calculate to reduce computational time. Feel free to adjust the list
# the reactions are: lactate uptake/production, phenylalanine hydroxylase 
# (which has an isoenzyme that performs a similar reaction), Pyruvate dehydrogenase, Succinate dehydrogenase 
# and ATPase
interesting_reactions = ['EX_lac_D_e','PHETHPTOX2','PDHm','SUCD1m','ATPS4mi',]

result = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=interesting_reactions)
result[['minimum','maximum']]

result.plot.bar(stacked=True)

<span style="color:red">**Assignment (3 min):**</span> Which of these are reversible? i.e. they can carry flux in either forward or backward direction while the cell is still capable of optimal growth rate. 

# Are essential amino acids essential?
We will now look into which reactions the model predicts to be essential for growth. Are the "essential" amino acids predicted to be essential?

## Calculating the essential genes using flux variability analysis
One of the uses of FVA is to predict essential reactions. If we require growth to be non-zero then reactions that are essential will have an associated FVA interval that does not contain zero.

<span style="color:red">**Assignment (3 min):**</span> Make sure you understand the statement made above. Then look back at the plot above and figure out which of the reactions visualized there is essential? 

A helpful example is to think of glucuse exchange reaction. What would happen to this flux when the cell tries to grow optimally? Does this reaction need to carry flux? What does that say about the FVA interval? 

The cell below returns a table with essential reactions predicted by FVA. It might take a while to run...

In [ ]:
model = M.copy()

# first perform pFBA to find a list of potentially essential reactions
# reasoning: if reaction v is essential it ALWAYS has to carry flux. So also in this pFBA solution. 
# doing this first allows us to perform FVA on a smaller set of reactions
sol = pfba(model)

# take only those reactions that carry flux
reactions_with_flux = sol.fluxes[sol.fluxes.abs() > 1e-9].index.tolist()

# perform fba on the flux carrying reactions
fva_sol = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=reactions_with_flux,fraction_of_optimum=1)

# filter out 
fva_sol = fva_sol.loc[(fva_sol['minimum'] > 1e-9) | (fva_sol['maximum'] < -1e-9) ]
df = fva_sol.copy()
for r in df.index.tolist():
    df = df.set_value(r,'reaction',model.reactions.get_by_id(r).reaction)
    df = df.set_value(r,'genes',model.reactions.get_by_id(r).gene_reaction_rule)
df

<span style="color:red">**Assignment (2 min):**</span> Search to see if the glucose exchange reaction is in the table above. Is it essential? What is the FVA interval allowed under the condition of optimality? 

<span style="color:red">**Assignment (5 min):**</span> Rerun the cell above but change the 'fraction_of_optimum' parameter to less than 1. 1 means exactly optimal. 0.8 would mean you allow the solution to be at least 80% of the optimal growth rate. etc. What happens to the glucose exchange reaction interval when you decrease the 'fraction_of_optimum' parameter? Is there a point where it becomes non-essential?

# Self-check questions
Make sure you are able to explain in your own words: 
1. What the two core components of the human metabolic reconstruction model are
2. How thermodynamics of reactions may be taken into account in the reaction bounds
3. What flux balance analysis is
4. What FBA actually computates
6. How FVA is different from FBA 
7. What you can use FVA for